In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
from enum import Enum, IntEnum
from bs4 import BeautifulSoup


In [12]:
url_to_scrap = ""

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(options=options)
driver.set_window_size(1400,1000)
driver.get("https://www.pepper.pl/nowe")
time.sleep(0.7)
page = driver.page_source
soup = BeautifulSoup(page, "html5lib")


In [30]:
from collections import Counter
articles = soup.find_all('article')
for article in articles:
    name = article.find_all(attrs={'class': "cept-tt thread-link linkPlain thread-title--list js-thread-title"})[0]['title']
    date_tag = article.find_all('div', {"class":"size--all-s flex boxAlign-jc--all-fe boxAlign-ai--all-c flex--grow-1 overflow--hidden"})
    discount_price = article.find_all(attrs={'class': "thread-price text--b cept-tp size--all-l size--fromW3-xl"})
    raw_string_list = date_tag[0].get_text(strip=True, separator='_').split('_')
    items_to_remove = list()
    filtered_list = list()
    for string in raw_string_list:
        if "/" in string:
            items_to_remove.append(string)
        if ":" in string:
            items_to_remove.append(string)
        if string in ["Jutro", "DZISIAJ", "Lokalnie"]:
            items_to_remove.append(string)
        if string.startswith("Wysyłka"):
            items_to_remove.append(string)
    counts = Counter(items_to_remove)
    for string in raw_string_list:
        if counts[string]:
            counts[string] -= 1
        else:
            filtered_list.append(string)


    print(f"discount price: {discount_price} date: {filtered_list}, name: {name}")
    #print(articles)

discount price: [<span class="thread-price text--b cept-tp size--all-l size--fromW3-xl">967,99zł</span>] date: ['3 min'], name: Zestaw Garmin Edge 530 Bundle z czujnikiem tętna, prędkość i kadencji.
discount price: [<span class="thread-price text--b cept-tp size--all-l size--fromW3-xl">79,17zł</span>] date: ['7 min'], name: Rick Astley - Hold Me in Your Arms (2023 Remaster) - Winyl LP
discount price: [<span class="thread-price text--b cept-tp size--all-l size--fromW3-xl">30,89zł</span>] date: ['10 min'], name: Watomierz miernik zużycia energii 16A
discount price: [<span class="thread-price text--b cept-tp size--all-l size--fromW3-xl">35,99zł</span>] date: ['10 min'], name: Wałek do ciasta piernik folkroll
discount price: [<span class="thread-price text--b cept-tp size--all-l size--fromW3-xl">259zł</span>] date: ['12 min'], name: Chłodzenie Cooler Master MasterLiquid ML240L V2 RGB 2x120mm (kolor biały w tej samej cenie) @ x-kom
discount price: [<span class="thread-price text--b cept-tp 

In [14]:
import enum
from typing import List, Union
from bs4 import BeautifulSoup, Tag



class Months(Enum):

    sty = '01'
    lut = '02'
    mar = '03'
    kwi = '04'
    maj = '05'
    cze = '06'
    lip = '07'
    sie = '08'
    wrz = '09'
    paz = '10'
    paź = '10'
    lis = '11'
    gru = '12'

    @classmethod
    def to_dict(cls):
        """Returns a dictionary representation of the enum."""
        return {e.name: e.value for e in cls}

    @classmethod
    def keys(cls):
        """Returns a list of all the enum keys."""
        return cls._member_names_

    @classmethod
    def values(cls):
        """Returns a list of all the enum values."""
        return list(cls._value2member_map_.keys())

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(options=options)
driver.set_window_size(1400,1000)
driver.get("https://www.pepper.pl/promocje/akcja-swiec-blaskiem-z-odblaskiem-darmowe-odblaski-w-warszawie-761817")
time.sleep(0.7)
page = driver.page_source
soup = BeautifulSoup(page, "html5lib")


date_string = soup.find_all('div', {"class":"space--mv-3"})[0].find('span')['title']
filtered_list = date_string.split()
day_string = filtered_list[0]
month_string = filtered_list[1]
year_string = filtered_list[2].strip(',')

if len(day_string[0]) == 2:
    day = day_string
else:
    day = day_string.zfill(2)

month = Months.__members__[month_string].value
year = year_string
prepared_data = '-'.join([year, month, day])

print(prepared_data)

2023-12-03


In [3]:
from typing import List, Union
from datetime import datetime, timedelta, date, timezone
from bs4 import BeautifulSoup, Tag
import sys
import csv
import os
import time
import logging
import re

import html5lib
import pandas as pd
import signal
import traceback
from source.pepper_app.constans import OLD_DATES_DATA_PATTERN_1, OLD_DATES_DATA_PATTERN_2

from source.pepper_app.environment_config import CustomEnvironment
from source.pepper_app.constans import DATA_HEADER, STATS_HEADER, REQUEST_HEADER, RESPONSE_HEADER

from collections import Counter
from requests.exceptions import ConnectionError, HTTPError, MissingSchema, ReadTimeout
from source.pepper_app.constans import (CSV_COLUMNS,
                                STATS_HEADER)

class ScrapPage:


    def __init__(self, category_type: str, articles_to_retrieve: int, to_csv: bool=False,
                to_database: bool=False, to_statistics: bool=True, searched_article: str='NA', 
                scrap_continuously: bool=False, scrap_choosen_data: bool=True) -> None:
        self.category_type = category_type
        self.articles_to_retrieve = articles_to_retrieve
        self.to_database = to_database
        self.to_csv = to_csv
        self.to_statistics = to_statistics
        self.start_page = 3
        self.searched_article = searched_article
        self.scrap_continuously = scrap_continuously
        self.scrap_choosen_data = scrap_choosen_data

    def scrap_page(self, url_to_scrap: str, driver: webdriver=None) -> BeautifulSoup:
        """Setting up selenium webdriver, scraping page with bs4."""
        try:
            options = Options()
            #options.add_argument("--headless")
            options.add_argument("--no-sandbox")
            if driver is None:
                driver = webdriver.Chrome(options=options)
                #driver = webdriver.Remote(command_executor='http://selenium-hub:4444/wd/hub', options=options)
            driver.set_window_size(1400,1000)
            driver.get(url_to_scrap)
            time.sleep(0.7)
            page = driver.page_source
            driver.quit()
            soup = BeautifulSoup(page, "html5lib")
            return soup
        except ConnectionError as e:
            raise ConnectionError(f"ConnectionError occured: {e}. \nTry again later")
        except MissingSchema as e:
            raise MissingSchema(f"MissingSchema occured: {e}. \nMake sure that protocol indicator is icluded in the website url")
        except HTTPError as e:
            raise HTTPError(f"HTTPError occured: {e}. \nMake sure that website url is valid")
        except ReadTimeout as e:
            raise ReadTimeout(f"ReadTimeout occured: {e}. \nTry again later")

    def select_url(self) -> str:
        """Selection of the website address depending on the type of scrapping."""
        if self.scrap_continuously == True:
            url_to_scrap = "".join([CustomEnvironment.get_url(), "nowe"])
            return url_to_scrap
        elif self.category_type == "nowe" and self.scrap_continuously == False:
            url_to_scrap = "".join([CustomEnvironment.get_url(), self.category_type, "?page=", str(self.start_page)])
            return url_to_scrap
        elif self.category_type == "search" and self.scrap_continuously == False:
            searched_article = str(self.searched_article.replace(" ","%20"))
            url_to_scrap = "".join([CustomEnvironment.get_url(), self.category_type, "?q=",
                                    searched_article, "&page=", str(self.start_page)])
            return url_to_scrap
        else:
            raise Exception(f"The variables were defined incorrectly.")


    def infinite_scroll_handling(self) -> List[str]:
        """Handling scraping through subsequent pages."""
        try:
            flag = True
            retrived_articles = list()
            while flag:
                url_to_scrap = self.select_url()
                soup = self.scrap_page(url_to_scrap)
                flag_nowe = CheckConditions(soup).check_if_last_page_nowe()
                flag_search = CheckConditions(soup).check_if_last_page_search()
                if flag_nowe == False or flag_search == False:
                    flag = False
                    return retrived_articles[:self.articles_to_retrieve]
                flag = CheckConditions(soup).check_if_no_items_found()
                if flag == False:
                    return retrived_articles[:self.articles_to_retrieve]
                if flag == True:
                    articles = soup.find_all('article')
                    retrived_articles += articles
                else:
                    return retrived_articles[:self.articles_to_retrieve]
                if len(retrived_articles) >= int(self.articles_to_retrieve):
                    flag = False
                    return retrived_articles[:self.articles_to_retrieve]
                self.start_page += 1
        except Exception as e:
            raise Exception(f"Infinite scroll failed:\
                            {e}\n Tracking: {traceback.format_exc()}")



    def get_items_details_depending_on_the_function(self):
        """Completing the list of articles and extracting data details depending on the type of scrapping."""
        if self.scrap_continuously == True and self.scrap_choosen_data == False:
            while True:
                retrived_articles = self.scrap_continuously_by_refreshing_page()
                self.get_items_details(retrived_articles)
        elif self.scrap_continuously == False and self.scrap_choosen_data == True:
            retrived_articles = self.infinite_scroll_handling()
            all_items = self.get_items_details(retrived_articles)
            return all_items
        else:
            raise Exception(f"Matching get_items_details depending on the selected \
                            functionality failed. \n Tracking: {traceback.format_exc()}")


    def get_items_details(self, retrived_articles) -> list():
        """Getting item detailes."""
        start_time = datetime.utcnow().replace(tzinfo=timezone.utc)
        all_items = list()
        try:
            for article in retrived_articles:
                item = list()
                item.append(GetItemId(article).get_data())
                item.append(GetItemName(article).get_data())
                item.append(GetItemDiscountPrice(article).get_data())
                item.append(GetItemPercentageDiscount(article).get_data())
                item.append(GetItemRegularPrice(article).get_data())
                item.append(GetItemAddedDate(article).get_data())
                item.append(GetItemUrl(article).get_data())
                if item not in all_items:
                    all_items.append(item)
                if '' in item:
                    logging.warning("Data retrieving failed. None values detected")
                    break
                if self.to_csv:
                    self.save_data_to_csv(item)
                #if self.to_database:
                #    LoadItemDetailsToDatabase(item).load_to_db()
        except Exception as e:
            logging.warning(f"Getting item details failed :\
                        {e}\n Tracking: {traceback.format_exc()}")

        end_time = datetime.utcnow().replace(tzinfo=timezone.utc)
        action_execution_datetime = end_time - start_time

        if self.to_statistics:
            try:
                stats_info = self.get_scraping_stats_info(action_execution_datetime)
                #LoadScrapingStatisticsToDatabase(stats_info).load_to_db()
            except Exception as e:
                logging.warning(f"Populating ScrapingStatistics table failed: {e}\n Tracking: {traceback.format_exc()}")

        return all_items

    def save_data_to_csv(self, item) -> None:
        """Saving data to csv file."""
        try:
            header = False
            if not os.path.exists('scraped.csv'):
                header = True
                df = pd.DataFrame([item], columns=CSV_COLUMNS)
                df.to_csv('scraped.csv', header=header, index=False, mode='a')
            else:
                header = False
                df_e = pd.read_csv('scraped.csv')
                df = pd.DataFrame([item], columns=CSV_COLUMNS)
                if df['item_id'][0] not in df_e['item_id'].tolist():
                    df.to_csv('scraped.csv', header=header, index=False, mode='a')
        except Exception as e:
            logging.warning(f"Saving data to csv failed: {e}\n Tracking: {traceback.format_exc()}")


    def get_scraping_stats_info(self, action_execution_datetime: datetime) -> List[Union[str, int, bool, float]]:
        """Getting scraping stats info."""
        stats_info = list()

        category_type = self.category_type
        retrieved_articles_quantity = self.articles_to_retrieve
        time_of_the_action = datetime.utcnow().replace(tzinfo=timezone.utc)
        action_execution_datetime = action_execution_datetime
        searched_article = self.searched_article
        to_csv = self.to_csv
        to_database  = self.to_database
        scrap_continuously = self.scrap_continuously
        scrap_choosen_data = self.scrap_choosen_data

        stats=[category_type, retrieved_articles_quantity,
            time_of_the_action, action_execution_datetime, searched_article,
            to_csv, to_database, scrap_continuously, scrap_choosen_data]

        for field in stats:
            stats_info.append(field)

        return stats_info


    def scrap_continuously_by_refreshing_page(self) -> List[str]:
        """Scraping data function for continuously scraping feature."""
        retrived_articles = list()

        soup = self.scrap_page()
        time.sleep(20)
        articles = soup.find_all('article')
        retrived_articles += articles

        return retrived_articles


class CheckConditions:


    def __init__(self, soup: BeautifulSoup) -> None:
        self.soup = soup


    def check_if_last_page_nowe(self) -> bool:
        """Checking 'nowe' category to verify if the scraped page is the last one."""
        try:
            searched_ending_string = self.soup.find_all('h1', {"class":"size--all-xl size--fromW3-xxl text--b space--b-2"})[0].get_text()
            if searched_ending_string.startswith("Ups"):
                logging.warning("No more pages to scrap.")
                return False
        except:
            return True

    def check_if_last_page_search(self) -> bool:
        """Checking 'search' category to verify if the scraped page is the last one."""
        try:
            searched_ending_string = self.soup.find_all('h3', {"class":"size--all-l"})[0].get_text()
            searched_articles_number = self.soup.find_all('span', {"class":"box--all-i size--all-s vAlign--all-m"})[0].get_text()
            searched_articles_number = int(searched_articles_number.replace(" ","").strip("\n\t Okazje()"))
            if searched_ending_string.startswith("Ups") and searched_articles_number > 0:
                logging.warning("No more pages to scrap.")
                return False
        except:
            return True

    def check_if_no_items_found(self) -> bool:
        """Checking if searched item was found."""
        try:
            searched_ending_string = self.soup.find_all('h3', {"class":"size--all-l"})[0].get_text()
            searched_articles_number = self.soup.find_all('span', {"class":"box--all-i size--all-s vAlign--all-m"})[0].get_text()
            searched_articles_number = int(searched_articles_number.replace(" ","").strip("\n\t Okazje()"))
            if searched_ending_string.startswith("Ups") and searched_articles_number == 0:
                logging.warning("The searched item was not found.")
                return False
        except:
            return True


class Months(Enum):

    sty = '01'
    lut = '02'
    mar = '03'
    kwi = '04'
    maj = '05'
    cze = '06'
    lip = '07'
    sie = '08'
    wrz = '09'
    paz = '10'
    paź = '10'
    lis = '11'
    gru = '12'

    @classmethod
    def to_dict(cls):
        """Returns a dictionary representation of the enum."""
        return {e.name: e.value for e in cls}

    @classmethod
    def keys(cls):
        """Returns a list of all the enum keys."""
        return cls._member_names_

    @classmethod
    def values(cls):
        """Returns a list of all the enum values."""
        return list(cls._value2member_map_.keys())


class GetItemName:

    def __init__(self, article: Tag) -> None:
        self.article = article

    def get_data(self) -> str:
        try:
            name = self.article.find_all(attrs={'class': "cept-tt thread-link linkPlain thread-title--list js-thread-title"})[0]['title']
            return name
        except IndexError as e:
            name = self.article.find_all(attrs={'class': "thread-link linkPlain thread-title--list js-thread-title"})[0]['title']
            return name
        except Exception as e:
            logging.warning(f"Getting item name failed: {e}\n Tracking: {traceback.format_exc()}")


class GetItemId:

    def __init__(self, article: Tag) -> None:
        self.article = article

    def get_data(self) -> int:
        try:
            item_id = self.article.get("id")
            item_id = item_id.strip('thread_')
            item_id = int(item_id)
            return item_id
        except Exception as e:
            logging.warning(f"Getting item id failed: {e}\n Tracking: {traceback.format_exc()}")


class GetItemDiscountPrice:

    def __init__(self, article: Tag) -> None:
        self.article = article

    def get_data(self) -> Union[float, str]:
        try:
            discount_price = self.article.find_all(attrs={'class': "thread-price text--b cept-tp size--all-l size--fromW3-xl"})
            if len(discount_price) == 0:
                discount_price = self.article.find_all(attrs={'class': "thread-price text--b cept-tp size--all-l size--fromW3-xl text--color-greyShade"})

            if len(discount_price) > 0:
                discount_price = discount_price[0].get_text().strip('zł').replace('.','').replace(',','.').replace(' ','')
                if discount_price == "ZADARMO":
                    discount_price = float(0)
                else:
                    discount_price = float(discount_price)                
            else:
                """The attribute does not exist or the class name is invalid."""
                discount_price = "NA"
            return discount_price
        except Exception as e:
            logging.warning(f"Getting item discount price failed: {e}\n Tracking: {traceback.format_exc()}")



class GetItemRegularPrice:

    def __init__(self, article: Tag) -> None:
        self.article = article

    def get_data(self) -> Union[float, str]:
        try:
            regular_price = self.article.find_all(attrs={'class': "mute--text text--lineThrough size--all-l size--fromW3-xl"})
            if len(regular_price) > 0:
                regular_price = float(regular_price[0].get_text().strip('zł').replace('.','').replace(',','.').replace(' ',''))
            else:
                """The attribute does not exist or the class name is invalid."""
                regular_price = "NA"
            return regular_price
        except Exception as e:
            logging.warning(f"Getting item regular price failed: {e}\n Tracking: {traceback.format_exc()}")



class GetItemPercentageDiscount:

    def __init__(self, article: Tag) -> None:
        self.article = article

    def get_data(self) -> Union[float, str]:
        try:
            percentage_discount = self.article.find_all(attrs={'class': "space--ml-1 size--all-l size--fromW3-xl"})
            if len(percentage_discount) > 0:
                percentage_discount = float(percentage_discount[0].get_text().strip('%'))
            else:
                """The attribute does not exist or the class name is invalid."""
                percentage_discount = "NA"
            return percentage_discount
        except Exception as e:
            logging.warning(f"Getting item percentage discount failed: {e}\n Tracking: {traceback.format_exc()}")


class GetItemUrl:

    def __init__(self, article: Tag) -> None:
        self.article = article

    def get_data(self) -> str:
        try:
            item_url = self.article.find_all(attrs={'class', 'cept-tt thread-link linkPlain thread-title--list js-thread-title'})[0]['href']
            print(self.article)
            return item_url
        except IndexError as e:
            item_url = 'NA - Login Required.'
            return item_url
        except Exception as e:
            logging.warning(f"Getting item url failed: {e}\n Tracking: {traceback.format_exc()}")


class GetItemAddedDate:

    def __init__(self, article: Tag) -> None:
        self.article = article

    def get_raw_data(self) -> List[str]:

        try:
            date_tag = self.article.find_all('div', {"class":"size--all-s flex boxAlign-jc--all-fe boxAlign-ai--all-c flex--grow-1 overflow--hidden"})
            raw_string_list = date_tag[0].get_text(strip=True, separator='_').split('_')
            return raw_string_list
        except Exception as e:
            logging.warning(f"Getting item added date failed: {e}\n Tracking: {traceback.format_exc()}")

    def get_data(self) -> str:

        try:
            filtered_list = self.clean_list()
            filtered_list = self.check_missing_date()
            date_string_likely = filtered_list[0]
            if date_string_likely == "NA":
                url_with_item = GetItemUrl.get_data(self)
                soup = self.scrap_page(url_with_item)
                prepared_data = self.fill_missing_date(soup)
                return prepared_data
            else:
                stripped_date_string_likely = self.strip_date_string(date_string_likely)
                prepared_data = self.date_format_conversion(stripped_date_string_likely)
            return prepared_data

        except TypeError as e:
            raise TypeError(f"Invalid html class name: {e}")

    def strip_date_string(self, date_string_likely: str) -> str:

        try:
            if date_string_likely.startswith("Zaktualizowano ") and date_string_likely.endswith(" temu"):
                stripped_date_string_likely = date_string_likely.lstrip("Zaktualizowano ")
                stripped_date_string_likely = stripped_date_string_likely.rstrip(" temu")
                return stripped_date_string_likely
            elif date_string_likely.endswith("Lokalnie"):
                stripped_date_string_likely = date_string_likely.rstrip("Lokalnie")
                return stripped_date_string_likely
            else:
                stripped_date_string_likely = date_string_likely
                return stripped_date_string_likely
        except Exception as e:
            logging.error(f"Stripping date string failed: {e}\n Tracking: {traceback.format_exc()}")

    def date_format_conversion(self, stripped_date_string_likely: str) -> str:

        try:
            if stripped_date_string_likely.endswith(('min', 'g', 's')):
                prepared_data = date.today().strftime("%Y-%m-%d")
                return prepared_data
            elif stripped_date_string_likely.startswith(tuple(Months.keys())) and len(stripped_date_string_likely) < 8:
                if len(stripped_date_string_likely[4:]) == 3:
                    day = stripped_date_string_likely[4:6]
                else:
                    day = stripped_date_string_likely[4:5].zfill(2)
                month = Months.__members__[stripped_date_string_likely[0:3]].value
                year = str(date.today().year)
                prepared_data = '-'.join([year, month, day])
                return prepared_data
            elif bool(re.search(OLD_DATES_DATA_PATTERN_1, stripped_date_string_likely)):
                day = stripped_date_string_likely[4:6]
                month = Months.__members__[stripped_date_string_likely[0:3]].value
                year = stripped_date_string_likely[8:13]
                prepared_data = '-'.join([year, month, day])
                return prepared_data
            elif bool(re.search(OLD_DATES_DATA_PATTERN_2, stripped_date_string_likely)):
                day = stripped_date_string_likely[4:5].zfill(2)
                month = Months.__members__[stripped_date_string_likely[0:3]].value
                year = stripped_date_string_likely[7:12]
                prepared_data = '-'.join([year, month, day])
                return prepared_data
        except Exception as e:
            logging.error(f"Data format conversion tailed: {e}\n Tracking: {traceback.format_exc()}")


    def clean_list(self) -> List[str]:

        raw_string_list = self.get_raw_data()
        items_to_remove = list()
        filtered_list = list()

        try:
            for string in raw_string_list:
                if "/" in string:
                    items_to_remove.append(string)
                if ":" in string:
                    items_to_remove.append(string)
                if string in ["Jutro", "DZISIAJ", "Lokalnie", "Stacjonarnie"]:
                    items_to_remove.append(string)
                if string.startswith("Wysyłka"):
                    items_to_remove.append(string)

            counts = Counter(items_to_remove)

            for string in raw_string_list:
                if counts[string]:
                    counts[string] -= 1
                else:
                    filtered_list.append(string)

            return filtered_list

        except TypeError as e:
            raise TypeError(f"Input data must be a list: {e}")


    def check_missing_date(self) -> List[str]:

        filtered_list = self.clean_list()

        try:
            if len(filtered_list) == 0:
                filtered_list.append("NA")
                return filtered_list
            else:
                return filtered_list
        except TypeError as e:
            raise TypeError(f"Input data must be a list: {e}")


    def fill_missing_date(self, soup) -> str:

        try:
            date_string = soup.find_all('div', {"class":"space--mv-3"})[0].find('span')['title']
            filtered_list = date_string.split()
            day_string = filtered_list[0]
            month_string = filtered_list[1]
            year_string = filtered_list[2].strip(',')

            if len(day_string[0]) == 2:
                day = day_string
            else:
                day = day_string.zfill(2)

            month = Months.__members__[month_string].value
            year = year_string
            prepared_data = '-'.join([year, month, day])
            return prepared_data
        except TypeError as e:
            raise TypeError(f"Input data must be a list: {e}")


    def scrap_page(self, url_with_item: str, driver: webdriver=None) -> BeautifulSoup:

        try:
            options = Options()
            options.add_argument("--headless")
            options.add_argument("--no-sandbox")
            if driver is None:
                driver = webdriver.Chrome(options=options)
                #driver = webdriver.Remote(command_executor='http://selenium-hub:4444/wd/hub', options=options)
            driver.get(url_with_item)
            time.sleep(0.7)
            page_with_item = driver.page_source
            driver.quit()
            soup = BeautifulSoup(page_with_item, 'html5lib')
            return soup
        except ConnectionError as e:
            raise ConnectionError(f"ConnectionError occured: {e}. \nTry again later")
        except MissingSchema as e:
            raise MissingSchema(f"MissingSchema occured: {e}. \nMake sure that protocol indicator is icluded in the website url")
        except HTTPError as e:
            raise HTTPError(f"HTTPError occured: {e}. \nMake sure that website url is valid")
        except ReadTimeout as e:
            raise ReadTimeout(f"ReadTimeout occured: {e}. \nTry again later")

category_type = "search"
articles_to_retrieve = 5
searched_article = "S22"

output = ScrapPage(category_type, articles_to_retrieve, searched_article=searched_article)
all_items = output.get_items_details_depending_on_the_function()


for i in all_items:
    print(i)


[707151, 'Smartfon Samsung Galaxy S22 8/128GB Zielony', 2439.0, 'NA', 'NA', '2023-08-09', 'NA - Login Required.']
[704584, 'Smartfon Samsung Galaxy S22 (S901) 8/256GB 6,1" Dynamic AMOLED 2X 2340x1080 3700mAh Dual SIM 5G Green', 2799.0, 'NA', 'NA', '2023-08-02', 'NA - Login Required.']
[702868, 'Galaxy S21 Case / obudowa / etui', 17.1, 'NA', 'NA', '2023-07-28', 'NA - Login Required.']
[702788, '2 szt. Wodoszczelne etui na telefon komórkowy TOPK nowy model 2023 - TYLKO PRIME', 39.99, 'NA', 'NA', '2023-07-28', 'NA - Login Required.']
[700810, 'Białe Etui Samsung Silicone Cover With Strap do Galaxy S22 Ultra', 19.99, -86.0, 139.99, '2023-07-21', 'NA - Login Required.']
